In [1]:
import requests 
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '6971dee46a92', 'cluster_name': 'docker-cluster', 'cluster_uuid': 't3P3CbdtTRy27stLS-rqaQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:20<00:00, 46.02it/s]


In [ ]:
user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

## Q3. Searching

In [ ]:
response = es.search(index=index_name, body=search_query)
# print(response['hits']['hits'][0]['_score'])

for hit in response['hits']['hits']:
    doc = hit['_score']
    print(doc)
    

75.66571
75.66571
44.00519
44.00519
38.7155


In [ ]:
user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

## Q4. Filtering

In [ ]:
response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overridin...

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overridin...

Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker con...



In [ ]:
def retrieve_documents(query, index_name="course-questions", max_results=3):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [ ]:
user_question = "How do I execute a command in a running docker container?"
response = retrieve_documents(user_question)

for doc in response:
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overridin...

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overridin...

Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker con...



## Q5. Building a prompt

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}
CONTEXT:
{context}
""".strip()

In [ ]:
context_docs = retrieve_documents(user_question)

context_result = ""

for doc in context_docs:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()
print(len(context))

1155


In [ ]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [ ]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

## Q6. Tokens

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")
print(len(encoding.encode(prompt)))

325
